In [ ]:
%pip install python-terrier
%pip install repro-eval
%pip install ranx

In [1]:
import pandas as pd
import pyterrier as pt
if not pt.started():
    pt.init()

bm25_t3 = pt.io.read_results('runs/CIR_BM25.t3')
bm25_t4 = pt.io.read_results('runs/CIR_BM25.t4')
bm25_t5 = pt.io.read_results('runs/CIR_BM25.t5')
bm25_t4_extended = pd.read_csv('runs/CIR_BM25_extended.t4', sep=' ')

bm25_t4_strip = bm25_t4_extended[bm25_t4_extended['queryid_t5'].notna() & bm25_t4_extended['docid_t5'].notna()]
bm25_t4_strip['queryid']= bm25_t4_strip['queryid_t5']
bm25_t4_strip['docid']= bm25_t4_strip['docid_t5']
bm25_t4_strip = bm25_t4_strip[['queryid', 'docid', 'relevance', 'score', 'run']]
bm25_t4_strip = bm25_t4_strip.rename(columns={'queryid': 'qid', 'docid': 'docno', 'relevance': 'rank', 'run': 'name'})

qids = set(bm25_t5['qid']).intersection(set(bm25_t4_strip['qid']))
bm25_t5 = bm25_t5[bm25_t5['qid'].isin(qids)]

/Users/timo/code/CLEF2024-LongEval-CIR/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
/var/folders/tb/5vs85b2520s950r538g7zznr0000gn/T/ipykernel_18784/4119560510.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bm25_t4_strip['queryid']= bm25_t4_strip['queryid_t5']
/var/folders/tb/5vs85b2520s950r538g7zznr0000gn/T/ipykernel_18784/4119560510.py:13: SettingWithCopyWar

In [7]:
import numpy as np
from repro_eval.Evaluator import RpdEvaluator


def time_fuse(run_recent, run_old, _lambda=0.5):
    qid_ranking_groups = run_old.groupby('qid')
    qid_ranking_dict = {qid: list(ranking['docno']) for qid, ranking in qid_ranking_groups}
    
    def weigh(row):
        if row['docno'] in qid_ranking_dict.get(row['qid']):
            return row['score'] * _lambda ** 2
        else:
            return row['score'] * (1-_lambda) ** 2              
    reranking = run_recent.copy()
    
    reranking['score'] = reranking.groupby('qid')['score'].transform(lambda x : x / x.max())
    reranking['score'] = reranking.progress_apply(weigh, axis=1)
    reranking = reranking.sort_values(['qid','score'], ascending=False).groupby('qid').head(10000)
    reranking['rank'] = reranking.groupby('qid')['score'].rank(ascending=False).astype(int)
    return reranking

# _lambda = 0.5 + 1e-3
_lambda = 0.5 + 1e-12
bm25_t5_reranked_by_t4 = time_fuse(bm25_t5, bm25_t4_strip, _lambda=_lambda)
pt.io.write_results(bm25_t5_reranked_by_t4, 'bm25_t5_reranked_by_t4', format='trec',run_name='bm25_t5_reranked_by_t4')
pt.io.write_results(bm25_t5, 'bm25_t5', format='trec',run_name='bm25_t5')
rpd_eval = RpdEvaluator(run_b_orig_path='bm25_t5', run_b_rep_path='bm25_t5_reranked_by_t4')
correlations = rpd_eval.ktau_union().get('baseline')
correlation_scores = [x for x in list(correlations.values()) if ~np.isnan(x)]
print("Avg. Kendall's tau: ", sum(correlation_scores) / len(correlation_scores))

100%|██████████| 216218/216218 [00:02<00:00, 103813.55it/s]


Avg. Kendall's tau:  0.9049362485729147


In [12]:
import numpy as np
from repro_eval.Evaluator import RpdEvaluator
from ranx import Run, fuse

def filter_and_fuse(run_recent, old_runs: list):
    qid_ranking_groups = run_recent.groupby('qid')
    qid_ranking_dict_recent = {qid: pd.Series(ranking['score'].values, ranking['docno']).to_dict() for qid, ranking in qid_ranking_groups}
    runs = [Run.from_dict(qid_ranking_dict_recent)]
    for run_old in old_runs:
        qid_ranking_groups = run_old.groupby('qid')
        qid_ranking_dict_old = {qid: pd.Series(ranking['score'].values, ranking['docno']).to_dict() for qid, ranking in qid_ranking_groups}
        for qid, ranking in qid_ranking_dict_old.items():
            docs_recent = qid_ranking_dict_recent.get(qid).keys()
            qid_ranking_dict_old[qid] = {docid: score for docid, score in ranking.items() if docid in docs_recent}
        runs.append(Run.from_dict(qid_ranking_dict_old))
    combined_run = fuse(runs = runs, method = "rrf")

    return combined_run

run = filter_and_fuse(bm25_t5, [bm25_t4_strip])
run.save('bm25_t5_rrf_t4', kind='trec')
rpd_eval = RpdEvaluator(run_b_orig_path='bm25_t5', run_b_rep_path='bm25_t5_rrf_t4')
correlations = rpd_eval.ktau_union().get('baseline')
correlation_scores = [x for x in list(correlations.values()) if ~np.isnan(x)]
print("Avg. Kendall's tau: ", sum(correlation_scores) / len(correlation_scores))

Avg. Kendall's tau:  0.004377672896374601
